In [32]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

/home/octav/gitrepos/tum-thesis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time

In [34]:
os.getcwd()

'/home/octav/gitrepos/tum-thesis'

In [35]:
data_loader = get_loader("iclr19-graph2graph/data/qed", "val", 48, False)

In [36]:
for i in data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break
    
print(len(X.mols))
print(sum([y for (_, y) in X.scope]))
print(sum([y**2 for (_, y) in X.scope]))
print(sum([y for (_, y) in Y.scope]))
print(sum([y**2 for (_, y) in Y.scope]))

48
1121
26779
1022
22052


In [37]:
sys.argv = ["", "-cuda", "-pred_hidden", "150"]
args = get_args()
args.output_dir = "mol_opt/output/"
# args.device = "cpu"

In [8]:
gcn = GCN(args).to(args.device)
opt = torch.nn.Linear(50, 50).to(args.device)

In [9]:
gcn

GCN(
  (W_message_i): Linear(in_features=100, out_features=50, bias=False)
  (W_message_h): Linear(in_features=50, out_features=50, bias=False)
  (W_message_o): Linear(in_features=143, out_features=50, bias=True)
  (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
  (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
  (dropout_gcn): Dropout(p=0.0, inplace=False)
  (dropout_ffn): Dropout(p=0.0, inplace=False)
)

In [10]:
gcn

GCN(
  (W_message_i): Linear(in_features=100, out_features=50, bias=False)
  (W_message_h): Linear(in_features=50, out_features=50, bias=False)
  (W_message_o): Linear(in_features=143, out_features=50, bias=True)
  (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
  (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
  (dropout_gcn): Dropout(p=0.0, inplace=False)
  (dropout_ffn): Dropout(p=0.0, inplace=False)
)

In [11]:
opt(gcn.forward(Y)[0])

tensor([[ 0.0079,  0.0743, -0.1079,  ...,  0.1204,  0.2161,  0.1556],
        [ 0.0249,  0.0846, -0.0451,  ...,  0.0463,  0.1724,  0.2085],
        [-0.0030,  0.1260,  0.0180,  ...,  0.1078,  0.1869,  0.1830],
        ...,
        [ 0.0186,  0.0592, -0.0904,  ...,  0.1361,  0.2348,  0.1759],
        [ 0.0481,  0.1973, -0.1271,  ...,  0.0720,  0.2060,  0.1390],
        [ 0.0014,  0.1280, -0.1103,  ...,  0.0922,  0.2469,  0.1206]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [12]:
x_embedding = gcn.forward(X)[0]
stx, lex = X.scope[0]
x1 = x_embedding.narrow(0, stx, lex).view(lex, 1, -1).repeat(1, lex, 1)
x2 = x_embedding.narrow(0, stx, lex).view(1, lex, -1).repeat(lex, 1, 1)
_bonds = torch.cat((x1, x2), dim = 2)
# bonds_logits = 

In [13]:
torch.cat

<function _VariableFunctions.cat>

In [14]:
print (np.cumsum(([len(x.atoms) for x in X.mols])))

[  18   42   61   87  106  130  150  167  187  212  237  262  286  308
  331  353  375  394  418  437  457  477  500  526  552  575  598  622
  643  663  688  711  735  757  778  799  816  842  862  884  905  929
  950  969  993 1017 1039 1061]


In [15]:
x = X.mols[0]

In [16]:
x.atoms[0].symbol

'C'

In [31]:
x.bonds[0].bond_type

NameError: name 'x' is not defined

In [33]:
args.pred_hidden

100

In [18]:
from mol_opt import mol_opt
molopt = mol_opt.MolOpt(args).to(device = args.device)
molopt

MolOpt(
  (GCN): GCN(
    (W_message_i): Linear(in_features=100, out_features=50, bias=False)
    (W_message_h): Linear(in_features=50, out_features=50, bias=False)
    (W_message_o): Linear(in_features=143, out_features=50, bias=True)
    (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
    (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
    (dropout_gcn): Dropout(p=0.0, inplace=False)
    (dropout_ffn): Dropout(p=0.0, inplace=False)
  )
  (opt0): Linear(in_features=50, out_features=50, bias=True)
  (opt1): Linear(in_features=50, out_features=50, bias=True)
)

In [19]:
# molopt.align(molopt.encode(X), X, molopt.encode(Y), Y)

In [20]:
x_embedding = molopt.encode(X)
x_embedding.shape

torch.Size([1920, 50])

In [21]:
from mol_opt.ot_utils import compute_barycenter
compute_barycenter(x_embedding.narrow(0,0,20), 20, num_iters=50)

tensor([[ 1.5640e-01,  2.1761e-01,  1.8304e-01, -1.6272e-01,  3.1493e-02,
          1.1379e-01,  2.1669e-02,  1.8854e-01, -2.9571e-01,  1.8985e-02,
         -1.4275e-01,  1.9132e-01,  2.1506e-02,  6.2977e-02, -5.7108e-02,
          4.4616e-02,  1.1306e-01, -2.1863e-01, -1.5550e-02, -4.0034e-01,
          1.6929e-01,  1.9568e-01, -1.9850e-01, -2.8040e-02,  5.4685e-02,
         -1.1141e-01, -2.1223e-01, -1.0030e-01, -1.0619e-01, -1.0559e-01,
         -1.7550e-01, -1.1165e-01,  1.1228e-01, -1.1534e-02,  3.5927e-01,
         -7.8190e-02, -2.3244e-01, -1.4523e-01,  1.9819e-01,  1.3657e-01,
         -3.0067e-01,  7.3870e-03, -8.7360e-02,  2.0422e-01, -2.5709e-01,
         -9.0651e-02,  9.8129e-03,  1.8269e-01, -2.4321e-01,  6.7296e-02],
        [-1.0911e-01, -6.2154e-02, -1.5407e-02,  3.6712e-02,  7.0942e-03,
          3.2297e-01, -1.5229e-01, -3.8390e-02, -1.7681e-01,  7.7100e-03,
         -1.1580e-01, -2.4396e-02,  1.2572e-01,  2.1479e-01, -1.9864e-01,
         -1.0103e-02, -5.2607e-02,  9

In [22]:
# these numbers should all be very small
# that means that the computation converged
xhat = np.sort(compute_barycenter(x_embedding.narrow(0,0,20), 20, num_iters=50)[:,0].cpu().detach().numpy())
xreal = np.sort(x_embedding.narrow(0,0,20)[:,0].cpu().detach().numpy())
print (xhat)
print (xreal)
np.linalg.norm(xhat - xreal)

[-0.35099155 -0.25470862 -0.19054031 -0.11130951 -0.10911432 -0.10428502
 -0.09701747 -0.09069885 -0.0901949  -0.08784354 -0.01179874 -0.01083078
  0.01919781  0.02007846  0.02709519  0.05596866  0.1076645   0.11048253
  0.12628663  0.15640064]
[-0.35099155 -0.25470862 -0.19054031 -0.11130951 -0.10911432 -0.10428501
 -0.09701747 -0.09069885 -0.0901949  -0.08784354 -0.01179874 -0.01083078
  0.01919781  0.02007846  0.02709519  0.05596866  0.1076645   0.11048252
  0.12628663  0.15640062]


1.8273868e-08

In [23]:
from mol_opt.decoder_mol_opt import MolOptDecoder
molopt_decoder = MolOptDecoder(args).to(device = args.device)
molopt_decoder

MolOptDecoder(
  (fc1_SYMBOLS): Linear(in_features=50, out_features=100, bias=True)
  (fc2_SYMBOLS): Linear(in_features=100, out_features=64, bias=True)
  (fc1_CHARGES): Linear(in_features=50, out_features=100, bias=True)
  (fc2_CHARGES): Linear(in_features=100, out_features=5, bias=True)
  (fc1_BONDS): Linear(in_features=100, out_features=100, bias=True)
  (fc2_BONDS): Linear(in_features=100, out_features=5, bias=True)
)

In [14]:
res = molopt_decoder.forward(x_embedding, Y)
res_discrete = molopt_decoder.discretize(*res)

In [27]:
res_discrete

(tensor([54, 29, 54,  ..., 19, 29, 29], device='cuda:0',
        grad_fn=<NotImplemented>),
 tensor([4, 4, 4,  ..., 4, 4, 4], device='cuda:0', grad_fn=<NotImplemented>),
 tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0', grad_fn=<NotImplemented>))

In [32]:
from mol_opt.ot_utils import encode_target
encode_target(Y)

(tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         ...,
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.]]))

In [23]:
save_model(molopt, args, args.output_dir, "test")

Model saved to: mol_opt/output//model_test


In [24]:
args.output_dir

'mol_opt/output/'

In [25]:
# for epoch in range(args.n_epochs):
#     start = time.time()
#     for idx, i in enumerate(data_loader):
#         X = (MolGraph(i[0]))
#         Y = (MolGraph(i[1]))
# 
#         # create your optimizer
#         optimizer = torch.optim.SGD(molopt.parameters(), lr=0.01)
# 
#         # in your training loop:
#         optimizer.zero_grad()   # zero the gradient buffers
#         loss = molopt.forward_train(X, Y)
#         loss.backward()
#         optimizer.step()    # Does the update
# 
#         print ("Iter: {}, loss: {}".format(idx, loss.item()))
#     end = time.time()
#     print("Time for epoch {}: {}", epoch, end - start)

In [26]:
X.scope

[(0, 18),
 (18, 24),
 (42, 19),
 (61, 26),
 (87, 19),
 (106, 24),
 (130, 20),
 (150, 17),
 (167, 20),
 (187, 25),
 (212, 25),
 (237, 25),
 (262, 24),
 (286, 22),
 (308, 23),
 (331, 22),
 (353, 22),
 (375, 19),
 (394, 24),
 (418, 19),
 (437, 20),
 (457, 20),
 (477, 23),
 (500, 26),
 (526, 26),
 (552, 23),
 (575, 23),
 (598, 24),
 (622, 21),
 (643, 20),
 (663, 25),
 (688, 23),
 (711, 24),
 (735, 22),
 (757, 21),
 (778, 21),
 (799, 17),
 (816, 26),
 (842, 20),
 (862, 22),
 (884, 21),
 (905, 24),
 (929, 21),
 (950, 19),
 (969, 24),
 (993, 24),
 (1017, 22),
 (1039, 22)]

In [138]:
x_embedding.narrow(0, 0, 22)

tensor([[-0.0455,  0.0447,  0.0261,  ..., -0.0160,  0.0526,  0.0204],
        [-0.0465,  0.0597,  0.0177,  ..., -0.1785, -0.0618,  0.0922],
        [ 0.0161,  0.0673, -0.1091,  ...,  0.0081, -0.0677,  0.1738],
        ...,
        [ 0.0850, -0.0669, -0.0290,  ..., -0.0392, -0.0300,  0.0311],
        [ 0.1073, -0.0826, -0.0462,  ..., -0.0227, -0.0654,  0.0300],
        [ 0.0796,  0.0683,  0.0362,  ..., -0.0448, -0.0863,  0.0600]],
       device='cuda:0', grad_fn=<SliceBackward>)

In [144]:
x_embedding[0:22,:]

torch.Size([22, 50])

In [136]:
x_embedding.narrow(0, 22, 18)

tensor([[-9.2475e-02,  4.4592e-02,  4.1157e-02, -6.2958e-02, -2.6449e-02,
          5.1672e-03, -4.6384e-02, -5.3797e-02, -7.3464e-02, -7.3005e-02,
         -1.7704e-02,  8.3490e-02, -1.3573e-01,  8.2092e-03, -5.4852e-02,
         -1.0005e-01,  1.0265e-01,  8.8467e-02,  1.8191e-02,  8.6351e-02,
          1.7423e-01,  2.4751e-02,  6.3913e-03,  3.5624e-03,  8.4073e-03,
         -2.3538e-02,  1.3579e-01,  4.8769e-02,  1.5200e-01, -2.5218e-02,
          2.4999e-01, -7.4404e-03,  6.2257e-02, -1.2847e-01, -1.4130e-01,
          1.4734e-01, -2.1842e-03,  8.3792e-02, -8.7801e-02,  4.2352e-02,
          2.5093e-01, -3.2312e-02,  7.7363e-02,  6.2747e-02,  1.1126e-01,
          6.4715e-02, -3.6868e-02,  1.1756e-03,  7.7242e-03,  7.6595e-03],
        [ 2.0119e-03, -1.1420e-01, -6.6094e-03, -6.3875e-02,  8.7873e-02,
          1.7288e-02,  9.8871e-02, -1.4052e-01, -5.1341e-02, -1.6861e-01,
         -7.1023e-02,  3.3994e-02,  2.1864e-02, -9.5770e-03,  4.5321e-02,
          1.1468e-01,  1.7144e-01,  1

In [35]:
# start = time.time()
# for idx, i in enumerate(data_loader):
#     X = (MolGraph(i[0]))
#     Y = (MolGraph(i[1]))
#     
#     # create your optimizer
#     optimizer = torch.optim.SGD(molopt.parameters(), lr=0.01)
# 
#     # in your training loop:
#     optimizer.zero_grad()   # zero the gradient buffers
#     loss = molopt.forward_train(X, Y)
#     loss.backward()
#     optimizer.step()    # Does the update
# 
#     print (idx, loss)
# end = time.time()
# print(end - start)

# run the implemented function

In [28]:
sys.argv = ["", "-cuda", "-pred_hidden", "150"]
args = get_args()
args.n_epochs = 40 
args.output_dir = "mol_opt/output"
args.init_model = "transformer1"
args.init_decoder_model = "{}_decode".format(args.init_model)
args

Namespace(N_transformer=6, agg_func='sum', batch_norm=False, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, ffn_activation='LeakyReLU', init_decoder_model='transformer1_decode', init_model='transformer1', linear_out=False, n_epochs=40, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=50, n_labels=1, n_layers=5, ot_solver='emd', output_dir='mol_opt/output', pc_hidden=50, pred_hidden=150, sinkhorn_entropy=0.1, sinkhorn_max_it=10000)

In [27]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train", 96, False)
val_data_loader = get_loader("iclr19-graph2graph/data/qed", "val", 96, False)

In [69]:
get_latest_model("gcn3_decode", args.output_dir)

No model gcn3_decode found in mol_opt/output! Starting from scratch.


(None, 0)

In [29]:
molopt = main(args, train_data_loader = train_data_loader, val_data_loader = val_data_loader)

No model transformer1 found in mol_opt/output! Starting from scratch.
No model transformer1_decode found in mol_opt/output! Starting from scratch.
Epoch: 1


/home/octav/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:394: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]


 train_mse: 51.106947
 val_mse: 39.266362
Epoch duration: 1527.9214224815369
Model saved to: mol_opt/output/model_transformer1_1
Model saved to: mol_opt/output/model_transformer1_decode_1
Epoch: 2
 train_mse: 39.063239
 val_mse: 38.922148
Epoch duration: 1613.0044691562653
Model saved to: mol_opt/output/model_transformer1_2
Model saved to: mol_opt/output/model_transformer1_decode_2
Epoch: 3
 train_mse: 38.706692
 val_mse: 38.638293
Epoch duration: 1651.5140421390533
Model saved to: mol_opt/output/model_transformer1_3
Model saved to: mol_opt/output/model_transformer1_decode_3
Epoch: 4
 train_mse: 38.733420
 val_mse: 38.737589
Epoch duration: 1577.0327999591827
Model saved to: mol_opt/output/model_transformer1_4
Model saved to: mol_opt/output/model_transformer1_decode_4
Epoch: 5


KeyboardInterrupt: 

In [42]:
import cProfile
cProfile.run('main(args, train_data_loader = train_data_loader, val_data_loader = val_data_loader)')

Epoch: 2
 train_mse: 40.374824
 val_mse: 38.866998
Epoch duration: 1921.0867624282837
Model saved to: mol_opt/output/model_transformer0_2
Model saved to: mol_opt/output/model_transformer0_decode_2
         888227500 function calls (886352703 primitive calls) in 1921.114 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   408871    0.314    0.000    4.459    0.000 <__array_function__ internals>:2(amax)
  7655076    3.786    0.000   34.072    0.000 <__array_function__ internals>:2(any)
  2888831    1.585    0.000    8.998    0.000 <__array_function__ internals>:2(atleast_1d)
  4499472    1.707    0.000    9.646    0.000 <__array_function__ internals>:2(concatenate)
  1998380    1.117    0.000    4.955    0.000 <__array_function__ internals>:2(copyto)
  2914091    1.306    0.000    4.903    0.000 <__array_function__ internals>:2(dot)
   441530    0.400    0.000    0.787    0.000 <__array_function__ internals>:2(empty_like)
  18

# try out some results

In [14]:
for i in val_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [15]:
x_embedding, x_delta = molopt.forward(X)
y_embedding = molopt.encode(Y)
y_aligned = molopt.align(x_embedding, X, y_embedding, Y)
xhat_delta = molopt.delta(x_embedding, y_aligned)

In [16]:
x_embedding.shape

torch.Size([2098, 50])

In [17]:
x_delta

tensor([[ 51.2291, -93.8777,  67.3637,  ..., -48.3670, -35.8315,  35.6972],
        [ 31.9506, -89.4450,  59.7837,  ..., -90.8423, -70.3604,  44.9571],
        [ 38.3420, -81.8791,  54.3903,  ..., -79.0527, -53.9882,  40.6473],
        ...,
        [ 31.7734,  -5.8973,  50.6671,  ..., -31.8489, -61.3425,  43.7533],
        [ 42.1679, -35.2635,  58.6213,  ..., -89.3948, -68.3651,  42.4910],
        [ 35.0003, -31.2854,  65.7140,  ..., -27.6888, -69.8974,  40.5666]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [21]:
x_embedding

tensor([[ 3.2541,  0.9079,  2.0818,  ..., -0.4530, -0.6870, 10.7181],
        [-4.7226,  2.0234,  3.6004,  ..., -7.1243, -5.1279,  7.1974],
        [-0.2887, -1.2837, 11.8498,  ..., -2.4560,  0.2970,  4.6080],
        ...,
        [ 0.2980, -3.5470,  1.3268,  ..., -1.1102, -1.8826,  6.5500],
        [-1.6307, -0.2229,  1.7089,  ..., -4.2314,  0.2962,  3.2936],
        [-1.9074, -1.6668, -0.7295,  ...,  3.5026, -6.8949,  4.3016]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [19]:
x_delta - xhat_delta

tensor([[  49.3381,  -93.0658,   71.5300,  ...,  -47.5423,  -28.6862,
           35.0504],
        [  27.5109,  -83.0824,   59.8990,  ..., -106.6606,  -69.4454,
           43.3924],
        [  44.8429,  -84.7408,   60.8294,  ...,  -75.3584,  -52.8182,
           41.3485],
        ...,
        [  31.7734,   -5.8973,   50.6671,  ...,  -31.8489,  -61.3425,
           43.7533],
        [  42.1679,  -35.2635,   58.6213,  ...,  -89.3948,  -68.3651,
           42.4910],
        [  35.0003,  -31.2854,   65.7140,  ...,  -27.6888,  -69.8974,
           40.5666]], device='cuda:0', grad_fn=<SubBackward0>)

In [20]:
molopt.delta(x_embedding, y) - xhat

NameError: name 'y' is not defined

In [35]:
Y.mols[]